# SVM

Support Vector Machine의 약자

이름대로 Vector 개념을 사용


데이터 프레임의 각 열은 열 벡터 형태로 구현되며, 열 벡터들이 각각 고유의 축을 갖는 벡터 공간을 만들어, 분석 대상이 되는 개별 관측값은 모든 열 벡터에 관한 값을 해당 축의 좌표로 표시하여 벡터 공간에서 위치를 나타냄

열 벡터가 2개 존재하는 데이터셋은 2차원 평면 공간에 좌표로 표시하고, 속성이 3개이면 3차원 공간에 표시


ㄴ프 모형은 학습을 통해 벡터 공간을 나누는 경계를 찾음

In [13]:
import pandas as pd
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics 

In [2]:
df = sns.load_dataset('titanic')

In [3]:
rdf = df.drop(['deck', 'embark_town'], axis=1)  
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)

In [7]:
features = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

In [9]:
onehot_sex = pd.get_dummies(features['sex'])
features = pd.concat([features, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(features['embarked'], prefix='town')
features = pd.concat([features, onehot_embarked], axis=1)

features.drop(['sex', 'embarked'], axis=1, inplace=True)

In [10]:
X=features[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C', 'town_Q', 'town_S']]  #독립 변수 X
y=features['survived']                      #종속 변수 Y

In [11]:
X = preprocessing.StandardScaler().fit(X).transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 
print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


## SVM Model

#### 모델 객체 생성 (kernel='rbf' 적용)

sklearn에서 가져온 svm 모듈의 svc() 함수를 사용하여 모델 객체를 생성

이때 데이터를 벡터 공간으로 매핑하는 함수를 커널이라 하는데

일단은 'rbf' 옵션으로 함수를 적용한다

rbf : radical basis function (방사형 기저 함수)

이외에도 Linear, Polynimial, Sigmoid 등의 커널 옵션을 줄 수 있다

In [41]:
svm_model = svm.SVC(kernel='rbf')

In [42]:
svm_model.fit(X_train, y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [43]:
y_hat = svm_model.predict(X_test)

In [44]:
svm_report = metrics.classification_report(y_test, y_hat)

print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



In [45]:
r_square = svm_model.score(X_test, y_test)
print(r_square)

0.813953488372093
